### Importing libraries

In [19]:
from azure.cosmos import CosmosClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.cosmos.partition_key import PartitionKey

from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents import SearchClient  

from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

import pandas as pd

from dotenv import load_dotenv
import pandas as pd
load_dotenv(override=True)
import json

import os

# Using DefaultAzureCredential (recommended)
# https://techcommunity.microsoft.com/t5/azure-architecture-blog/configure-rbac-for-cosmos-db-with-managed-identity-instead-of/ba-p/3056638#:~:text=Create%20custom%20roles%20MyReadOnlyRole%20and%20MyReadWriteRole%20with%20both,definition%20create%20-a%20%24accountName%20-g%20%24resourceGroupName%20-b%20%40role-definition-ro.json
aad_credentials = DefaultAzureCredential()

AZURE_COSMOS_DB_ENDPOINT=os.environ['AZURE_COSMOS_DB_ENDPOINT']
AZURE_COSMOS_DB_KEY= os.environ['AZURE_COSMOS_DB_KEY']
AZURE_COSMOS_DB_DATABASE= os.environ['AZURE_COSMOS_DB_DATABASE']
AZURE_COSMOS_DB_CONN= os.environ['AZURE_COSMOS_DB_CONN']
azurecosmosdbclient = CosmosClient(AZURE_COSMOS_DB_ENDPOINT, credential=aad_credentials)
BING_API_KEY =  os.environ['BING_API_KEY']

CONTAINER_ID = os.environ['AZURE_COSMOS_DB_CONTAINER']
PartitionKeyPath = "/chunk_id"

database_client = azurecosmosdbclient.get_database_client(AZURE_COSMOS_DB_DATABASE)
container_client = database_client.get_container_client(CONTAINER_ID)
azure_openai_chatgpt_deployment= 'gpt4o'

from openai import AzureOpenAI
aoai_client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-07-01-preview"
)

# Variables not used here do not need to be updated in your .env file
AZURE_SEARCH_SERVICE_ENDPOINT = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
AZURE_SEARCH_ADMIN_CREDENTIAL = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME").lower().replace("_", "-")

search_client = SearchClient(endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=AZURE_SEARCH_ADMIN_CREDENTIAL)


### CosmosDB Tool calling

#### CosmosDB GetCosmosDBAnswer Function definition

In [2]:
def GetCosmosDBAnswer(question):

    prompt_template = """You are a SQL programmer Assistant. Your role is to generate CosmosDB No SQL code (CosmosDB NoSQL) to retrieve an answer to a natural language query. Make sure to disambiguate column names when creating queries that use more than one table. If a valid SQL query cannot be generated, only say "ERROR:" followed by why it cannot be generated.
                  Do not answer any questions on inserting or deleting rows from the table. Instead, say "ERROR: I am not authorized to make changes to the data".

                  Use the following database schema to write CosmosDB NoSQL queries:
                  {}(page_num INTEGER, content VARCHAR, title VARCHAR, chunk_id VARCHAR, preprocessing_pipeline VARCHAR, filename VARCHAR, filing_period VARCHAR, filing_date VARCHAR, form_type VARCHAR, ticker VARCHAR PRIMARY KEY (chunk_id))

                  ## Relevant comments abuout the data
                  ** This data is already chunked, meaning that the same file might have different chunks, each with a different chunk_id. The chunk_id is the primary key for this table. So if you wanted to know how many filings have been made, you would count the number of unique filename.
                  ** both filing_period and filing date are in the format YYYY-MM-DD
                  ** ticker is the stock ticker of the company that made the filing
                  ** form_type contains information related to SEC form type, for example 10K, 10Q. This fields should help when determining whether to use quarterly or yearly reports

                  ## Additional guidance
                  ** Please only answer questions that can be answered with the data provided. Do not make any assumptions about the data.
                  ** Please write the code for a CosmosDB NoSQL query
                  ** Please make sure that you use the right DISTINCT 
                  ** Always use dates in the format YYYY-MM-DD
                  ** If you need to ask any clafication, please ask for it in the format JSON format with the key: CLARIFICATION
                  ** If asked about any date, default to filing_date unless the question implies that is a question about the filing period.
                  ** Dates should be explicity stated in the query. For example, if you are asked about filings in 2023, you should use the following format: '2023-01-01' and '2023-12-31'. Otehrwise, ask for the specific period.
                  ** Provide the answer in a JSON format with the key: NoSQLquery

                  ## Sample queries

                  BETWEEN Clasue:
                    ** Statement: You can use the BETWEEN keyword with a WHERE clause to express queries that filters results against ranges of string or numerical values. For example, the following query returns all items in which the price is between 17.25 and 25.50, again inclusive.
                    ** Specific intructions: always put the BETWEEN clause in parenthesis (c.price BETWEEN 17.25 AND 25.50)
                    ** NoSQL Query: SELECT * FROM c WHERE (c.price BETWEEN 17.25 AND 25.50)

                 DISTINCT and COUNT Clause:
                   ** Statement: You can combine the DISTINCT and COUNT keywords to return the number of unique items in a result set. For example, the following query returns the number of unique values of the price field.
                   ** NoSQL Query: SELECT VALUE COUNT(1) FROM (SELECT DISTINCT c.price FROM c)
                 
                 Additional query:
                    ** Explanation: For this query, the index matches any item that has a tag with a name of either "winter" or "fall", at least one quantity between zero and ten, and at least one warehouse where the backstock is false. The JOIN expression here performs the cross-product of all items of tags, onHandQuantities, and warehouseStock arrays for each matching item before any filter is applied. The WHERE clause then applies the filter predicate on each <c, t, n, s> tuple. For instance, if a matching item had ten items in each of the three arrays, it expands to 1 x 10 x 10 x 10 (that is, 1,000) tuples. Using subqueries here can help in filtering out joined array items before joining with the next expression.
                    ** query: SELECT VALUE COUNT(1) FROM products p JOIN t in p.tags JOIN q in p.onHandQuantities JOIN s in p.warehouseStock WHERE t.name IN ("winter", "fall") AND (q.quantity BETWEEN 0 AND 10) AND NOT s.backstock

                 Sample Query:
                    ** question: How many filings exist for 2023 on this container?
                    ** query: "SELECT VALUE COUNT(1) FROM (SELECT DISTINCT c.filename FROM c WHERE (c.filing_date BETWEEN '2023-01-01' AND '2023-12-31'))"
                
                questions: """.format(CONTAINER_ID)
    
    messages = [{"role":"system","content":prompt_template}, 
               {"role":"user","content":question}]

    response = aoai_client.chat.completions.create(model="gpt4o",  
                                        messages = messages, 
                                        temperature=0.1,  
                                        max_tokens=2000,
                                        response_format={ "type": "json_object" },
                                        seed = 42)
    SQLschema = response.choices[0].message.content
    
    if "ERROR" in SQLschema:
        print('ERROR')
        return str({"NoSQLQuery":"No query", "Answer": str(json.loads(SQLschema)['ERROR'])})
    elif 'CLARIFICATION' in SQLschema:
        print('CLARIFICATION')
        return str({"NoSQLQuery":"No query", "Answer": str(json.loads(SQLschema)['CLARIFICATION'])})
    else:
        NoSQLQuery = json.loads(SQLschema)['NoSQLquery']
        print(NoSQLQuery)
        return str({"NoSQLQuery":NoSQLQuery, "Answer": str([i for i in container_client.query_items(query=NoSQLQuery, enable_cross_partition_query=True)])})

#### RAG Tool GetRAGAnswer Function definition

In [3]:
#https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/advanced-workflow/query-rewrite/query-rewrite.ipynb

In [4]:
import json
                                                               
REWRITE_PROMPT = """You are a helpful assistant. You help users search for the answers to their questions.
You have access to Azure AI Search index with 100's of documents. Rewrite the following question into multiple to find the most relevant documents.

- If the "Current user question" has multiple questions, please generate search intents for all questions in a single array.
    - Always include a query for combined search intent. This extra search query will ensure we can find if a document exists that can answer question directly.
    - For example if a user asks - "What is A, B and C?", you should return - ["intent A", "intent B", intent C", "intent A, B and C"].

- Important information:
    - This system is designed to help with SEC filings, so if you find any company mentioned, please replace the company name that you find with ticker. For example, microsfot should be replaced with MSFT.

Always output a JSON object in the following format:
===
Input: "scalable storage solution"
Output: { "queries": ["what is a scalable storage solution in Azure", "how to create a scalable storage solution", "steps to create a scalable storage solution"] }
===
"""

# If you are not using a supported model or region, you may not be able to use json_object response format
# Please see https://learn.microsoft.com/azure/ai-services/openai/how-to/json-mode
def rewrite_query(question: str):
    response = aoai_client.chat.completions.create(
        model=azure_openai_chatgpt_deployment,
        seed=42,
        temperature=0.2,
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": REWRITE_PROMPT},
            {"role": "user", "content": f"Input: {question}"}
        ]
    )
    try:
        return json.loads(response.choices[0].message.content)
    except json.JSONDecodeError as e:
        print("JSON decoding error:", e)
        raise

In [5]:
rewrite_query("what is the best AOAI model")

{'queries': ['what is the best AOAI model',
  'comparison of AOAI models',
  'features of the best AOAI model',
  'advantages of the best AOAI model']}

In [6]:
def HybridSearch(question):
    vector_query = VectorizableTextQuery(text=question, k_nearest_neighbors=30, fields="content_vector", exhaustive=True)
    
    results = search_client.search(
        search_text=question,  
        vector_queries=[vector_query],
        query_type=QueryType.SEMANTIC, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
        top=5)
    
    data = [[result["id"], result["title"], result["content"], result["@search.score"], result["@search.reranker_score"],result['filename']] for result in results]
    return pd.DataFrame(data, columns=["id", "title", "content", "@search.score", "@search.reranker_score",'filename'])

In [7]:
import json
                                                               
# If you are not using a supported model or region, you may not be able to use json_object response format
# Please see https://learn.microsoft.com/azure/ai-services/openai/how-to/json-mode
def GetRAGAnswer(question: str):

    RAG_PROMPT = """You are an agent that works with SEC filings
    ## Very Important Instruction
        ### On Your Ability to Refuse Answering Out-of-Domain Questions
        - **Read the user's query, conversation history, and retrieved documents sentence by sentence carefully.**
        - Try your best to understand the user's query (prior conversation can provide more context, you can know what "it", "this", etc., actually refer to; ignore any requests about the desired format of the response), and assess the user's query based solely on provided documents and prior conversation.
        - Classify a query as 'in-domain' if, from the retrieved documents, you can find enough information possibly related to the user's intent which can help you generate a good response to the user's query. Formulate your response by specifically citing relevant sections.
        - For queries not upheld by the documents, or in case of unavailability of documents, categorize them as 'out-of-domain'.
        - You have the ability to answer general requests (**no extra factual knowledge needed**), e.g., formatting (list results in a table, compose an email, etc.), summarization, translation, math, etc. requests. Categorize general requests as 'in-domain'.
        - You don't have the ability to access real-time information, since you cannot browse the internet. Any query about real-time information (e.g., **current stock**, **today's traffic**, **current weather**), MUST be categorized as an **out-of-domain** question, even if the retrieved documents contain relevant information. You have no ability to answer any real-time query.
        - Think twice before you decide whether the user's query is really an in-domain question or not. Provide your reason if you decide the user's query is in-domain.
        - If you have decided the user's query is an in-domain question, then:
            * You **must generate citations for all the sentences** which you have used from the retrieved documents in your response.
            * You must generate the answer based on all relevant information from the retrieved documents and conversation history.
            * You cannot use your own knowledge to answer in-domain questions.
        - If you have decided the user's query is an out-of-domain question, then:
            * Your only response is "The requested information is not available in the retrieved data. Please try another query or topic."
        - For out-of-domain questions, you **must respond** with "The requested information is not available in the retrieved data. Please try another query or topic."

        ### On Your Ability to Do Greeting and General Chat
        - **If the user provides a greeting like "hello" or "how are you?" or casual chat like "how's your day going", "nice to meet you", you must answer with a greeting.
        - Be prepared to handle summarization requests, math problems, and formatting requests as a part of general chat, e.g., "solve the following math equation", "list the result in a table", "compose an email"; they are general chats. Please respond to satisfy the user's requirements.

        ### On Your Ability to Answer In-Domain Questions with Citations
        - Examine the provided JSON documents diligently, extracting information relevant to the user's inquiry. Forge a concise, clear, and direct response, embedding the extracted facts. Attribute the data to the corresponding document using the citation format [page_num]. Strive to achieve a harmonious blend of brevity, clarity, and precision, maintaining the contextual relevance and consistency of the original source. Above all, confirm that your response satisfies the user's query with accuracy, coherence, and user-friendly composition.
        - **You must generate a citation for all the document sources you have referred to at the end of each corresponding sentence in your response.**
        - **The citation mark [id] (for example: 10Q-MSFT-04-26-2022-chunk-id-47) must be placed at the end of the corresponding sentence which cited the document.**
        - **Every claim statement you generate must have at least one citation.**
        """
    
    retreival_df = HybridSearch(question)
    CONTEXT = retreival_df[['content', 'filename','id']].to_dict(orient='records')

    USER_QUESTION = "Please answer the following question with the conxt provided. Question: " + question + "\n" + "Context: " + str(CONTEXT)

    response = aoai_client.chat.completions.create(
        model=azure_openai_chatgpt_deployment,
        seed=42,
        temperature=0.2,
        messages=[
            {"role": "system", "content": RAG_PROMPT},
            {"role": "user", "content": USER_QUESTION}
        ]
    )
    return response.choices[0].message.content

In [8]:
question = ("How many stocks did Microsoft repurchased for the nine months ending march 2023?")
answer = GetRAGAnswer(question)

print("Answer: ", answer)
HybridSearch(question)

Answer:  Microsoft repurchased 55 million shares of its common stock for $13.8 billion during the nine months ended March 31, 2023 [10Q-MSFT-04-25-2023-chunk-id-46].


,id,title,content,@search.score,@search.reranker_score,filename
0,10Q-MSFT-04-26-2022-chunk-id-47,"**Item 2: Share Repurchases, Dividends, Off-Ba...",PART ! Item 2\nShare Repurchases\nFor the nine...,0.027313,3.404105,10Q-MSFT-04-26-2022
1,10Q-MSFT-04-25-2023-chunk-id-46,**Item 2. Management's Discussion and Analysis...,PART ! Item 2\nThe following table outlines th...,0.015873,3.365809,10Q-MSFT-04-25-2023
2,10K-MSFT-07-27-2023-chunk-id-52,**PART II Item 7**,PART II Item 7\nIncome Taxes\nAs a result of t...,0.018247,2.576517,10K-MSFT-07-27-2023
3,10Q-MSFT-04-25-2023-chunk-id-30,**Item 1: Revenue by Product and Service Offer...,"PART ! Item 1\nRevenue, classified by signific...",0.013158,2.457031,10Q-MSFT-04-25-2023
4,10Q-MSFT-04-25-2023-chunk-id-26,**Item 1: Share Repurchase Programs and Divide...,PART ! Item 1\nWe repurchased the following sh...,0.027584,2.412416,10Q-MSFT-04-25-2023


#### Bing Tool GetBingAnswer Function definition

In [ ]:
def BingSearchCall(question):
    import requests
    import json

    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': BING_API_KEY,
    }

    params = {
        # Query parameter
        'q': question,
        # Request parameter
        'count': '10',
        'offset': '0',
        'mkt': 'en-us',
        'safeSearch': 'Moderate',
    }

    response = requests.get("https://api.bing.microsoft.com/v7.0/search", headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    data = [[result["displayUrl"], result["snippet"]] for result in search_results['webPages']['value']]
    return pd.DataFrame(data, columns=["SourceURL", "snippet"])[:3].to_dict(orient='records')


def GetBingAnswer(question: str):

    RAG_PROMPT = """You are an agent that uses results from the Bing Web Search API
    ## Very Important Instruction
        ### On Your Ability to Refuse Answering Out-of-Domain Questions
        - **Read the user's query, conversation history, and retrieved documents sentence by sentence carefully.**
        - Try your best to understand the user's query (prior conversation can provide more context, you can know what "it", "this", etc., actually refer to; ignore any requests about the desired format of the response), and assess the user's query based solely on provided documents and prior conversation.
        - Classify a query as 'in-domain' if, from the retrieved documents, you can find enough information possibly related to the user's intent which can help you generate a good response to the user's query. Formulate your response by specifically citing relevant sections.
        - For queries not upheld by the documents, or in case of unavailability of documents, categorize them as 'out-of-domain'.
        - You have the ability to answer general requests (**no extra factual knowledge needed**), e.g., formatting (list results in a table, compose an email, etc.), summarization, translation, math, etc. requests. Categorize general requests as 'in-domain'.
        - You don't have the ability to access real-time information, since you cannot browse the internet. Any query about real-time information (e.g., **current stock**, **today's traffic**, **current weather**), MUST be categorized as an **out-of-domain** question, even if the retrieved documents contain relevant information. You have no ability to answer any real-time query.
        - Think twice before you decide whether the user's query is really an in-domain question or not. Provide your reason if you decide the user's query is in-domain.
        - If you have decided the user's query is an in-domain question, then:
            * You **must generate citations for all the sentences** which you have used from the retrieved documents in your response.
            * You must generate the answer based on all relevant information from the retrieved documents and conversation history.
            * You cannot use your own knowledge to answer in-domain questions.
        - If you have decided the user's query is an out-of-domain question, then:
            * Your only response is "The requested information is not available in the retrieved data. Please try another query or topic."
        - For out-of-domain questions, you **must respond** with "The requested information is not available in the retrieved data. Please try another query or topic."

        ### On Your Ability to Do Greeting and General Chat
        - **If the user provides a greeting like "hello" or "how are you?" or casual chat like "how's your day going", "nice to meet you", you must answer with a greeting.
        - Be prepared to handle summarization requests, math problems, and formatting requests as a part of general chat, e.g., "solve the following math equation", "list the result in a table", "compose an email"; they are general chats. Please respond to satisfy the user's requirements.

        ### On Your Ability to Answer In-Domain Questions with Citations
        - Examine the provided JSON documents diligently, extracting information relevant to the user's inquiry. Forge a concise, clear, and direct response, embedding the extracted facts. Attribute the data to the corresponding document using the citation format [page_num]. Strive to achieve a harmonious blend of brevity, clarity, and precision, maintaining the contextual relevance and consistency of the original source. Above all, confirm that your response satisfies the user's query with accuracy, coherence, and user-friendly composition.
        - **You must generate a citation for all the document sources you have referred to at the end of each corresponding sentence in your response.**
        - **The citation mark [URl] (for example: 10Q-MSFT-04-26-2022-chunk-id-47) must be placed at the end of the corresponding sentence which cited the document.**
        - **If you have more than one source for the same sentence, only use one citation.**
        - **Every claim statement you generate must have at least one citation.**
        """
    
    CONTEXT = BingSearchCall(question)

    USER_QUESTION = "Please answer the following question with the conxt provided. Question: " + question + "\n" + "Context: " + str(CONTEXT)

    response = aoai_client.chat.completions.create(
        model=azure_openai_chatgpt_deployment,
        seed=42,
        temperature=0.2,
        messages=[
            {"role": "system", "content": RAG_PROMPT},
            {"role": "user", "content": USER_QUESTION}
        ]
    )
    return response.choices[0].message.content

In [63]:
GetBingAnswer("who won the world series in 2023?")

'The Texas Rangers won the World Series in 2023, clinching their first title with a 5-0 victory in Game 5 against the Arizona Diamondbacks [https://www.mlb.com/news/rangers-win-world-series-2023].'

### Multi-Agent Conversation

In [92]:
def run_conversation(question):
    # Initial user message
    messages = [{"role":"system","content":"You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls"},
                 {"role": "user", "content": question}] # Single function call

    # Define the function for the model
    tools = [
        {
            "type": "function",
            "function": {
                "name": "GetRAGAnswer",
                "description": """This function takes in a question and then calls an Azure AI Search Index to retreive the top N documents, then it calls the Azure OpenAI model to generate an answer based on the context provided by the search engine.", 
                This database contains chunks (sections) that were extracting from SEC filings. Mentioning company names or tickers should not trigger this function unless they ask something about the schema. An example of a question that cannot be answered by this function is: How many filings did microsoft have in 2023? The reason is because it would need to investivate a DataBase to do that.
                This function should only be used to answer RAG-like questions with information found in SEC filings, for example, 10Ks and 10Qs.""",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "question that needs to be asked the search index and the model. Please use the verbatin question that the user asked.",
                        },
                    },
                    "required": ["question"],
                },
            }
        },
                {
            "type": "function",
            "function": {
                "name": "GetCosmosDBAnswer",
                "description": """Take a query, run a NoSQL query on the CosmosDB database, and return the result. 
                Only answer questions from the following database schema: DI_Text_HTML_PageSplitter(page_num INTEGER, content VARCHAR, title VARCHAR, chunk_id VARCHAR, preprocessing_pipeline VARCHAR, filename VARCHAR, filing_period VARCHAR, filing_date VARCHAR, form_type VARCHAR, ticker VARCHAR PRIMARY KEY (chunk_id))
                This database contains chunks (sections) that were extracting from SEC filings. Mentioning company names or tickers should not trigger this function unless they ask something about the schema. An example of a question that cannot be answered by this function is: How many stocks did microsoft repurchased in 2023?
                This function should only be used to answer database-like questions.
                DO NOT use this function if the question is not related to the database schema provided.""",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                            "description": "question that needs to be answered using the CosmosDB database. Please pass in verbatim the question that the user asked.",
                        },
                    },
                    "required": ["question"],
                },
            }
        }
    ]

    # First API call: Ask the model to use the function
    response = aoai_client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        seed=42
    )

    # Process the model's response
    response_message = response.choices[0].message
    messages.append(response_message)


    print("Model's response:")  
    print(response_message.tool_calls)  

    # Handle function calls
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function call: {function_name}")  
            print(f"Function arguments: {function_args}") 

            if function_name == "GetRAGAnswer":
                function_response = GetRAGAnswer(
                    question=function_args.get("question")
                )
                print(f"Function response: {function_response}")
            
            elif function_name == "GetCosmosDBAnswer":
                function_response = GetCosmosDBAnswer(
                    question=function_args.get("question")
                )
                print(f"Function response: {function_response}")
            else:
                function_response = "Function not found"
            
        messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
    else:
        print("No tool calls were made by the model. Using Bing Search API.")

        function_response = GetBingAnswer(question)
        print(f"Function response: {function_response}")

        messages.append({"role": "system", "content": function_response})
    
    print(messages)
    final_response = aoai_client.chat.completions.create(
        model='gpt4o',
        messages=messages,
        seed=42
    )

    return final_response.choices[0].message.content

In [93]:
run_conversation("How many stocks did Microsoft repurchased for the nine months ending march 2023?")

Model's response:
[ChatCompletionMessageToolCall(id='call_zOaJCZshaaE9l0BpbQ7ye11b', function=Function(arguments='{"question":"How many stocks did Microsoft repurchase for the nine months ending March 2023?"}', name='GetRAGAnswer'), type='function')]
Function call: GetRAGAnswer
Function arguments: {'question': 'How many stocks did Microsoft repurchase for the nine months ending March 2023?'}
Function response: Microsoft repurchased 55 million shares of its common stock for $13.8 billion during the nine months ended March 31, 2023 [10Q-MSFT-04-25-2023-chunk-id-46].
[{'role': 'system', 'content': 'You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls'}, {'role': 'user', 'content': 'How many stocks did Microsoft repurchased for the nine mo

'Microsoft repurchased 55 million shares of its common stock for $13.8 billion during the nine months ended March 31, 2023.'

In [94]:
run_conversation("How many documents exist for 2023?")

Model's response:
[ChatCompletionMessageToolCall(id='call_pBUYTDllNqGHGc0Om1dvMXrk', function=Function(arguments='{"question":"How many documents exist for 2023?"}', name='GetCosmosDBAnswer'), type='function')]
Function call: GetCosmosDBAnswer
Function arguments: {'question': 'How many documents exist for 2023?'}
SELECT VALUE COUNT(1) FROM c WHERE (c.filing_date BETWEEN '2023-01-01' AND '2023-12-31')
Function response: {'NoSQLQuery': "SELECT VALUE COUNT(1) FROM c WHERE (c.filing_date BETWEEN '2023-01-01' AND '2023-12-31')", 'Answer': '[864]'}
[{'role': 'system', 'content': 'You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls'}, {'role': 'user', 'content': 'How many documents exist for 2023?'}, ChatCompletionMessage(content=None, role=

'There are 864 documents for 2023.'

In [95]:
run_conversation("Who won the world series in 2023?")

Model's response:
None
No tool calls were made by the model. Using Bing Search API.
Function response: The Texas Rangers won the World Series in 2023, clinching their first title with a 5-0 victory in Game 5 against the Arizona Diamondbacks [https://www.mlb.com/news/rangers-win-world-series-2023].
[{'role': 'system', 'content': 'You are an assistant that help answering questions from either an AI Search Index or a CosmosDB Database. Please only answer the questions if any of the tools provide it, If tool_calls=None please state that you dont have access to the source of that information and pass the message from the function calls'}, {'role': 'user', 'content': 'Who won the world series in 2023?'}, ChatCompletionMessage(content="I don't have access to the source of that information.", role='assistant', function_call=None, tool_calls=None), {'role': 'system', 'content': 'The Texas Rangers won the World Series in 2023, clinching their first title with a 5-0 victory in Game 5 against the 

'The Texas Rangers won the World Series in 2023, defeating the Arizona Diamondbacks.'